[View in Colaboratory](https://colab.research.google.com/github/nicolasmetallo/road-defect-tf-implementation/blob/master/zooniverse_to_tfrecord.ipynb)

# Download data from Zooniverse and convert to TFRecord

---
**Author:** Nicolas Metallo & Cristian Belussi. Last updated on August 15, 2018.

This page walks through the steps required to train an object detection model on a local machine. It assumes the reader has completed the following prerequisites:

- The Tensorflow Object Detection API
- A valid data set has been created. See this page for instructions on how to generate a dataset for the PASCAL VOC challenge or the Oxford-IIIT Pet dataset.
- A Object Detection pipeline configuration has been written. See this page for details on how to write a pipeline configuration.

# *1) Clone GitHub, download images from Zooniverse, and import annotation data*

---

Source: https://github.com/Streets-Data-Collaborative/squid-vision/tree/master/tensorflow

In [0]:
#=============== Import Libraries ===============#
import os
import numpy as np
import pandas as pd
import glob
import json
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import cv2
from PIL import Image

#=============== Git Clone ===============#
!git clone https://github.com/nicolasmetallo/road-defect-tf-implementation.git
home_dir = "/content"
os.chdir(os.path.join(home_dir, "road-defect-tf-implementation"))
!pip install -r requirements.txt

#=============== Split 'images' into train, val, test ===============#
!python3 build_dataset.py --data_dir='images' --output_dir='images'

# *2) Extract annotation, image label, and labelmap (.pbtxt) data and save into csv*

---

Source: https://storage.googleapis.com/openimages/web/download.html

## Data Formats
### Bounding Boxes
Each row defines one bounding box.

```
ImageID,Label,XMin,XMax,YMin,YMax
000026e7ee790996,/m/07j7r,0.071905,0.145346,0.206591,0.391306
000026e7ee790996,/m/07j7r,0.439756,0.572466,0.264153,0.435122
000026e7ee790996,/m/07j7r,0.668455,1.000000,0.000000,0.552825
000062a39995e348,/m/015p6,0.205719,0.849912,0.154144,1.000000
000062a39995e348,/m/05s2s,0.137133,0.377634,0.000000,0.884185
0000c64e1253d68f,/m/07yv9,0.000000,0.973850,0.000000,0.043342
0000c64e1253d68f,/m/0k4j,0.000000,0.513534,0.321356,0.689661
0000c64e1253d68f,/m/0k4j,0.016515,0.268228,0.299368,0.462906
0000c64e1253d68f,/m/0k4j,0.481498,0.904376,0.232029,0.489017
```
#### ImageID: 
The image this box lives in.

#### Label: 
The object class this box belongs to.

#### XMin, XMax, YMin, YMax: 
Coordinates of the box, in normalized image coordinates. XMin is in [0,1], where 0 is the leftmost pixel, and 1 is the rightmost pixel in the image. Y coordinates go from the top pixel (0) to the bottom pixel (1).

Circular annotations will be turned into rectangular annotations by adding and substranging the radius to (x,y) coordinates, e.g. Xmin = x-r

![alt text](https://i.stack.imgur.com/53E6N.png)

### Class Names
The class names in MID format can be converted to their short descriptions by looking into class-descriptions.csv:

```
/m/0pc9,Alphorn
/m/0pckp,Robin
/m/0pcm_,Larch
/m/0pcq81q,Soccer player
/m/0pcr,Alpaca
/m/0pcvyk2,Nem
/m/0pd7,Army
/m/0pdnd2t,Bengal clockvine
/m/0pdnpc9,Bushwacker
/m/0pdnsdx,Enduro
/m/0pdnymj,Gekkonidae
```
Note the presence of characters like commas and quotes. The file follows standard CSV escaping rules. e.g.:

```
/m/02wvth,"Fiat 500 ""topolino"""
/m/03gtp5,Lamb's quarters
/m/03hgsf0,"Lemon, lime and bitters"
```

In [0]:
#=============== Import Annotation Data ===============#
data = pd.read_csv('street-quality-identification-device-syr-classifications.csv') # csv export from Zooniverse
split = ["train","val"]

#=============== Write DataFrame & Save to CSV ===============#

# DEFAULT_COLUMNS = ['image_id', 'xmin', 'ymin', 'xmax', 'ymax', 'label']

for each in split:
  images = []
  imageList = glob.glob(os.path.join(os.getcwd(),"images",each,"*.jpg"))
  
  for image in range(len(imageList)):
    images.append(imageList[image].split('/')[5])
    
  row_data = {}
  df_list = []
    
  for index in range(len(data)):
    imageDict = json.loads(data.subject_data[index])
    imageName = str(list(imageDict.values())[0]['Filename'])
    
    if imageName in images:
      annotation = list(json.loads(data.annotations[index]))[0]['value']
      for bbox in range(len(annotation)):
        try:
          r = int(annotation[bbox]['r'])
        except:
          continue
        x = int(annotation[bbox]['x'])
        y = int(annotation[bbox]['y'])
        label = annotation[bbox]['tool_label']
      
        row_data = {"image_id": str(imageName.replace(".jpg","")), "label": label.split()[0], "xmin": int(x-r), "xmax": int(x+r), "ymin": int(y-r), "ymax": int(y+r)}
        df_list.append(row_data)
        
  bbox_data = pd.DataFrame(df_list)
  bbox_data = bbox_data[['image_id', 'xmin', 'ymin', 'xmax', 'ymax', 'label']]
  bbox_data.loc[bbox_data.xmin < 0, "xmin"] = 0
  bbox_data.loc[bbox_data.ymin < 0, "ymin"] = 0
  
  bbox_data.to_csv(os.path.join(os.getcwd(),"images",each,"labels.csv"), index = False)

# *3) Review image with bounding boxes*

---

Source: https://www.kaggle.com/alekseit/simple-bounding-boxes

In [0]:
#=============== Visualize annotated images ===============#
def visualize_annotated_bbox(index, split = 'train'):
  bbox_data = pd.read_csv(os.path.join(os.getcwd(),"images",split,"labels.csv"))
  images_dir = os.path.join(os.getcwd(),"images",split)
  if index > len(bbox_data):
    print("Choose a smaller index. Max is {}".format(len(bbox_data)))
    return
  img = cv2.imread(os.path.join(images_dir,str(bbox_data.loc[index]["image_id"])+".jpg"))
  color = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  #im = Image.open(images_dir + imageName)
  #width, height = im.size

  plt.figure()
  plt.imshow(color)
  rect = Rectangle((bbox_data.loc[index]["xmin"],bbox_data.loc[index]["ymin"]),(bbox_data.loc[index]["xmax"]-bbox_data.loc[index]["xmin"]),(bbox_data.loc[index]["ymax"]-bbox_data.loc[index]["ymin"]), fill=False, color="red")

  plt.axes().add_patch(rect)
  plt.title('Label = {}'.format(bbox_data.loc[index]["label"]))
  plt.show()

# *4) Convert images to TFRecord*

---

Source: 
- https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/oid_inference_and_evaluation.md
- https://riemenschneider.hayko.at/vision/dataset/

In [0]:
#=============== Convert using Luminoth (https://luminoth.ai) ===============#
!pip install luminoth
!lumi dataset transform --type csv --data-dir images --output-dir output_dir --split train --split val